In [5]:
import numpy as np

In [6]:
import pickle
with open('cropdata', 'rb') as f:
    data = pickle.load(f)
for key in data:
    print(key)

sad
happy
neutral


In [7]:
print(data['happy'].shape)
print(np.max(data['happy']))

(400, 7000)
224.042269943


In [8]:
# print(np.max(data['happy']))
# print(np.min(data['happy']))
# divy=np.max(data['happy'])-np.min(data['happy'])
# y=(data['happy']-np.min(data['happy']))/divy
# print(np.max(y),(np.min(y)))

In [9]:
# print(np.max(data['neutral']))
# print(np.min(data['neutral']))
# divx=np.max(data['neutral'])-np.min(data['neutral'])
# x=(data['neutral']-np.min(data['neutral']))/divx
# print(np.max(x),(np.min(x)))

In [31]:
from keras.layers import Input, Dense,Dropout
from keras.models import Model
from keras import regularizers

encoding_dim = 1024

input_img = Input(shape=(7000,))
# add a Dense layer with a L1 activity regularizer
#encoded = Dense(encoding_dim, activation='relu',
#                activity_regularizer=regularizers.l1(10e-5))(input_img)
encoded = Dense(1024, activation='relu',activity_regularizer=regularizers.l1(10e-5))(input_img)
# encoded=Dropout(0.2)(encoded)
# encoded = Dense(512, activation='relu')(encoded)
# encoded=Dropout(0.2)(encoded)
encoded = Dense(256, activation='relu',activity_regularizer=regularizers.l1(10e-5))(encoded)

decoded = Dense(256)(encoded)
# decoded=Dropout(0.2)(decoded)
# decoded = Dense(512)(decoded)
# decoded=Dropout(0.2)(decoded)
# decoded = Dense(1024)(decoded)
# decoded=Dropout(0.2)(decoded)
decoded = Dense(7000)(encoded)

autoencoder = Model(input_img, decoded)


In [32]:
from keras import optimizers
adam= optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
autoencoder.compile(optimizer=adam, loss='mean_squared_error')#from keras.datasets import mnist

In [33]:
x = data['neutral']
y = data['happy']

x_train = x[:350]
X_train = y[:350]
autoencoder.fit(X_train, X_train,
                epochs=10,
                batch_size=16,
                shuffle=True,
                validation_data=(y[350:], y[350:])
               )# encode and decode some digits
# note that we take them from the *test* set
# encoded_imgs = encoder.predict(x_test)
# decoded_imgs = decoder.predict(encoded_imgs)

Train on 350 samples, validate on 50 samples
Epoch 1/10
350/350 [==============================] - 3s - loss: 125707849.2949 - val_loss: 901.2738
Epoch 2/10
350/350 [==============================] - 2s - loss: 2355.7046 - val_loss: 914.2525
Epoch 3/10
350/350 [==============================] - 2s - loss: 1125.0212 - val_loss: 914.1016
Epoch 4/10
350/350 [==============================] - 2s - loss: 1123.4442 - val_loss: 911.6514
Epoch 5/10
350/350 [==============================] - 2s - loss: 1120.2942 - val_loss: 908.5964
Epoch 6/10
192/350 [===============>..............] - ETA: 1s - loss: 1121.3714

KeyboardInterrupt: 

In [19]:
xy = x[380].reshape((1, 7000))
# print(x[380].shape)
# print(xy.shape)
#encoded_imgs = encoder.predict(xy)
#print(encoded_imgs.shape)
#mfccs = decoder.predict(encoded_imgs)
mfccs=autoencoder.predict(xy)
# mfccs = mfccs.reshape((200, 25800//200))
#print(mfccs)
#print(np.min(data['happy']))
#print(mfccs.shape)

In [20]:
import librosa
# import numpy as np
from IPython.lib.display import Audio

def invlogamplitude(S):
    """librosa.logamplitude is actually 10_log10, so invert that."""
    return 10.0**(S/10.0)

# filename = "a.wav"
# y, sr = librosa.load(filename)

# Y = librosa.stft(y)
# mfccs = librosa.feature.mfcc(y, n_mfcc=100)
def recon(mfccs):
    sr = 22050

    n_mfcc = mfccs.shape[0]
    n_mel = 128
    dctm = librosa.filters.dct(n_mfcc, n_mel)
    n_fft = 2048
    mel_basis = librosa.filters.mel(sr, n_fft)

    bin_scaling = 1.0/np.maximum(0.0005, np.sum(np.dot(mel_basis.T, mel_basis),
    axis=0))

    recon_stft = bin_scaling[:, np.newaxis] * np.dot(mel_basis.T,
    invlogamplitude(np.dot(dctm.T, mfccs)))

    excitation = np.random.randn(mfccs.shape[1] * 510)
    E = librosa.stft(excitation)
    recon = librosa.istft(E/np.abs(E)*np.sqrt(recon_stft))

    print(mfccs.shape)
    return Audio(recon, rate=sr)


In [21]:
recon(data['neutral'][380].reshape((200, 35), order='F'))

(200, 35)


In [22]:
mxx = y[380].reshape((200, 35), order='F')
# mxx = mxx * divx
# mxx += np.min(data['neutral'])

recon(mxx)
# recon(mfccs)

(200, 35)


In [23]:
myy = mfccs.reshape((200, 35), order='F')
# myy = myy * divy
# myy += np.min(data['happy'])

recon(myy)

(200, 35)


In [39]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 7000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              7169024   
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_12 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              525312    
__________

In [16]:
model_json= autoencoder.to_json()
with open("autoencoder.json","w") as json_file:
    json_file.write(model_json)
    
autoencoder.save_weights("autoencoder.h5")